In [ ]:
from config.config import PROJECT_ID, DATASET_ID, TABLE_ID, BUCKET_NAME, FOLDER_PREFIX, MODEL_NAME, VERTEXAI_LOCATION
from vertexai.generative_models import GenerativeModel, GenerationConfig

from google.cloud import bigquery
import pandas as pd
import numpy as np

In [ ]:
def read_bigquery_table_to_pandas(project_id, dataset_id, table_id):
    """
    Reads a BigQuery table into a Pandas DataFrame.

    Parameters:
    project_id (str): GCP project ID
    dataset_id (str): BigQuery dataset ID
    table_id (str): BigQuery table ID

    Returns:
    pandas.DataFrame: A DataFrame containing the data from the BigQuery table.
    """
    try:
        # Construct a BigQuery client object
        client = bigquery.Client(project=project_id)
        
        # Define the full table path
        table_ref = f"{project_id}.{dataset_id}.{table_id}"
        
        # Query to select everything from the specified table
        query = f"SELECT * FROM `{table_ref}`"
        
        # Execute the query and convert to a Pandas DataFrame
        dataframe = client.query(query).to_dataframe()
    
        return dataframe
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()



In [ ]:
df = read_bigquery_table_to_pandas(PROJECT_ID, DATASET_ID, TABLE_ID)

In [ ]:
def describe_dataframe_types(df):
    """
    Returns detailed information about the columns of a DataFrame,
    specifically identifying columns containing arrays.

    Parameters:
    df (pandas.DataFrame): The DataFrame to describe.

    Returns:
    dict: A dictionary with column names as keys and type information as values.
    """
    column_info = {}
    for column in df.columns:
        column_data = df[column]
        if isinstance(df[column].iloc[0], np.ndarray):
            column_info[column] = f"Array <{column_data.dtype}>"
        else:
            column_info[column] = str(column_data.dtype)
    return column_info

In [ ]:
def sample_random_rows(df, n):
    """
    Samples n random rows from a DataFrame and returns them as a list.

    Parameters:
    df (pandas.DataFrame): The DataFrame to sample from.
    n (int): The number of rows to sample.

    Returns:
    list: A list containing the sampled rows.
    """
    return df.sample(n).to_dict(orient='records')

sample_random_rows(df, 2)

In [ ]:
def generate_visualization_code(df_name, schema, examples, user_query, model_name):
    """
    Generates Python code to visualize or provide results for a user query on a Pandas DataFrame.

    Parameters:
    df_name (str): The name of the DataFrame.
    schema (str): The schema of the DataFrame.
    random_rows (str): A string representation of 10 random rows of the DataFrame.
    user_query (str): The user's query.
    model_name (str): The name of the generative model to use.

    Returns:
    str: The generated Python code.
    """
    formatted_schema = "\n".join([str(field) for field in schema])
    formatted_examples = "\n".join([str(row) for row in examples])
    
    prompt = f"""You are an advanced Python data analyst.

    I have a Pandas DataFrame named `{df_name}`. The schema of the DataFrame is as follows:
    {schema}

    Here are some random rows of the DataFrame:
    {formatted_examples}

    Given the following user query: "{user_query}", generate only the Python code that uses Pandas to visualize or provide results for this query.
    Pay attention that string values in most columns most be changed to a consistent case before proper analysis. Example 'medical insurance' and 'Medical Insurance' reflect same values but might be found in different case letters in the data. Make sure to correct this when needed.
    Make sure the code prints enough context to the user to make it more interesting to analyze."""
    
    print(prompt)
    # Initialize the model
    model = GenerativeModel(model_name)

    # Generate the structured output
    response = model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=1,
        ),
    )

    code = response.text.strip()
    # Remove code block markers if present
    if code.startswith("```python") and code.endswith("```"):
        code = code[9:-3].strip()
    elif code.startswith("```") and code.endswith("```"):
        code = code[3:-3].strip()
    return code    

In [ ]:
code = generate_visualization_code("DF", describe_dataframe_types(df), sample_random_rows(df, 2), "What are top benefits mentionned in job listings?", MODEL_NAME)

In [ ]:
global schema
global df_sample
schema = describe_dataframe_types(df)
df_sample = sample_random_rows(df, 2)

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import List, Dict, Any, Annotated
from typing_extensions import TypedDict
from langgraph.types import Send
import operator
from langgraph.checkpoint.memory import MemorySaver


from IPython.display import Image, display

class InputState(TypedDict):
    query: str
    df_name: str
    model_name: str
    
class OutputState(TypedDict):
    results: str

class OverallState(InputState, OutputState):
    code: str

    
def generate_code_(state: OverallState):
    query = state["query"]
    df_name = state["df_name"]
    model_name = state["model_name"]
    
    code = generate_visualization_code("DF", describe_dataframe_types(df), sample_random_rows(df, 2), "What are top benefits mentionned in job listings?", MODEL_NAME)
    return {"code": code}

def exec_code_(state: OverallState):
    code = state["code"]
    try:
        exec(code)
        results = "Success"
    except Exception as e:
        results = "Error"

    return {"results": results}


def code_result(state: OverallState):
    result = state["results"]
    if result == "Error":
        return "retry"
    else:
        return "success"    
    

# Build the graph with input and output schemas specified
graph = StateGraph(OverallState, input=InputState, output=OutputState)                
# Adding nodes
graph.add_node(generate_code_)
graph.add_node(exec_code_)
    
graph.add_edge(START, "generate_code_")
graph.add_edge("generate_code_", "exec_code_")

graph.add_conditional_edges(
    "exec_code_",
    code_result,

    {
        "retry": "generate_code_",
        "success": END
    },
)


app = graph.compile()  # Compile the graph

try:
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
app.invoke({"query": "Graph the top countries in data", "model_name": MODEL_NAME, "df_name": "DF"})